In [1]:
from simulation_modules import cobranzas_fine_tuning_template as cft
import pandas as pd
import json
from string import Template
from pprint import pprint

Generate a fine-tuning dataset with a chat-completion template for Cobranzas

### Generate Random and Custom data for the fine tuning dataset
Generate the random and additional data needed in the conversation with an instance of ConversationDataGenerator. The aditional info will be stored inside the 'aditional' dictionary.

In [5]:
bank_name = "Banco Azteca"
agent_name = "Raúl"

aditional_data = {
        "user_identity_confirmation": "[hola; Sí, soy yo; Sí, con $gender_confirm; Sí, yo hablo; Sí, aquí estoy; Sí, en qué puedo ayudarle; Así es, soy $gender_confirm; Correcto, está hablando con $gender_confirm; Sí, soy $gender_confirm, ¿de qué se trata?; Le escucho, soy $gender_confirm; Sí, conmigo está hablando; Sí, ¿qué necesita?]",
        "user_agrees_to_pay_today": "[Perdón, olvidé la fecha límite, pero haré el pago hoy mismo; Me descuidé, pero ya tengo el dinero y voy a pagar hoy; Sí, justo hoy puedo hacer el pago; No se preocupe, estoy por salir a pagar ahora mismo; Hoy mismo haré el pago, se me había pasado; Ya tengo el monto listo y voy a pagarlo en cuanto pueda; Sí, salgo en este momento a pagar; Voy a pagar en un rato, lo prometo; Ya tengo el dinero reunido, haré el pago hoy; Sí, voy de camino a realizar el pago, discúlpeme]"
}


data_generator = cft.ConversationDataGenerator(bank_name, agent_name, aditional_data)

# see a random sample of the data
conversation_data = data_generator.gen_data()
pprint(conversation_data)

{'aditional': {'user_agrees_to_pay_today': '[Perdón, olvidé la fecha límite, '
                                           'pero haré el pago hoy mismo; Me '
                                           'descuidé, pero ya tengo el dinero '
                                           'y voy a pagar hoy; Sí, justo hoy '
                                           'puedo hacer el pago; No se '
                                           'preocupe, estoy por salir a pagar '
                                           'ahora mismo; Hoy mismo haré el '
                                           'pago, se me había pasado; Ya tengo '
                                           'el monto listo y voy a pagarlo en '
                                           'cuanto pueda; Sí, salgo en este '
                                           'momento a pagar; Voy a pagar en un '
                                           'rato, lo prometo; Ya tengo el '
                                           'dinero reunido

### See the default placeholders and define a function to update them
View the default sub entry dictionary structure that contains the placeholders with a conversation_data sample. Note that there are no placeholders for 'user_identity_confirmation' or 'user_agrees_to_pay_today'. To add them we can define a function that return the extraction of all the information that will contain the new placeholders following the structure of the 'conversation_data' dict. Note that can be placeholders inside the new information (like '$gender_confirm'), so don't forget to add all the placeholders inside the function. In this case 'gender_confirm' is already in the default placeholders.

In [13]:
# print the default placeholders
sub_entry = cft.ConversationFactory.generate_basic_sub_entry(conversation_data)
pprint(sub_entry)

{'address': 'Calle 94 # 6-9 Sur\n051323\nFredonia, Antioquia',
 'agent': 'Raúl',
 'amount': 498826.19,
 'bank_name': 'Banco Azteca',
 'contact_number': 4775006675,
 'days': 24,
 'email': 'escobarmagdalena@gmail.com',
 'full_name': 'Néstor Henry Bohórquez Parra',
 'gender_confirm': 'él',
 'identification': '17496114',
 'known': 'lo',
 'phone_number': '9707304',
 'plural_ndays': 'días',
 'pronoun': 'el',
 'system_date_time': 'viernes 2024-12-20 02:03 ',
 'title': 'Señor',
 'today_date': '20 de diciembre de 2024',
 'tomorrow_date': '21 de diciembre de 2024'}


In [14]:
# function to update placeholders
def update_sub_entry(cdata):
    sub = {
        # default placeholders
        'bank_name': cdata['cobranzas']['bank_name'],
        'agent': cdata['cobranzas']['agent'],
        'contact_number': cdata['cobranzas']['contact_number'],
        'full_name': cdata['debtor']['full_name'],
        'address': cdata['debtor']['address'],
        'phone_number': cdata['debtor']['phone_number'],
        'email': cdata['debtor']['email'],
        'identification': cdata['debtor']['identification'],
        'days': cdata['debt']['ndays'],
        'plural_ndays': "días" if cdata['debt']['ndays'] > 1 else "día",
        'amount': cdata['debt']['amount'],
        'today_date': cdata['debt']['today_date'],
        'tomorrow_date': cdata['debt']['tomorrow_date'],
        'title': cdata['debtor']['title'],
        'pronoun': cdata['debtor']['pronoun'],
        'gender_confirm': cdata['debtor']['confirmation'],
        'known': cdata['debtor']['known'],
        'system_date_time': cdata['debt']['system_date_time'],
        # new placeholders
        'user_identity_confirmation': cdata['aditional']['user_identity_confirmation'],
        'user_agrees_to_pay_today': cdata['aditional']['user_agrees_to_pay_today']
    }

    return sub

### Define an instance of the ConversationFactory

To create an instance we must have:

- template file path, 
- system prompt str
- 'data_generator' instance of the ConversationDataGenerator
- 'update_sub_entry' func (if aditional information was added to the ConversationDataGenerator instance)
- use_random_variations (optional - use it to create the test dataset)
 
With The option 'use_random_variations' factory will randomly choose an option in the user part of the template instead of create an entire separated conversation example for each element in the user part of the template. If the template has 12 conversation fluxes the output will be 12 random user conversations. Later with the option 'dataset_size' of the method 'generate_fine_tuning_dataset' the number of test examples will be multiplied with this integer.

Withouth 'use_randdom_variations' the number of training examples is equal to the combinatorics of the placeholders per conversation flux inside the template.

In [15]:
 # generate a variational dataset
template_csv_path = 'simulation_modules/chat_flux_experimental.csv'

system_prompt_default = (
        "Tu nombre es $agent y trabajas para Cumplir SAS. " 
        "Tu tarea es comunicarte con los clientes con alta empatía y comprensión. "
        "Nombre Banco: $bank_name. "
        "Nombre Cliente: $full_name. " 
        "Monto Adeudado: $amount pesos. "
        "Fecha y hora de hoy: $system_date_time. "
        "Días de atraso en el pago: $days. " 
        "Fecha de pago máxima: $tomorrow_date. "
        "Número de contacto de Banco Azteca: $contact_number"
)

factory = cft.ConversationFactory(
        input_csv = template_csv_path,
        system_prompt = system_prompt_default,
        gen_data_instance = data_generator,
        sub_entry = update_sub_entry,
        use_random_variations=False               
)

In [16]:
conversations = factory.generate_fine_tuning_dataset()
print(f"Total variational training conversations: {len(conversations)}")

Total variational training conversations: 900


In [19]:
print(conversations[0])

{'messages': [{'role': 'system', 'content': 'Tu nombre es Raúl y trabajas para Cumplir SAS. Tu tarea es comunicarte con los clientes con alta empatía y comprensión. Nombre Banco: Banco Azteca. Nombre Cliente: Iván Tulio García Suárez. Monto Adeudado: 715763.29 pesos. Fecha y hora de hoy: viernes 2024-12-20 02:03 . Días de atraso en el pago: 5. Fecha de pago máxima: 21 de diciembre de 2024. Número de contacto de Banco Azteca: 4775006675'}, {'role': 'user', 'content': 'hola'}, {'role': 'assistant', 'content': 'Hola soy Raúl de Banco Azteca ¿me comunico con Iván Tulio García Suárez?'}, {'role': 'user', 'content': 'hola'}, {'role': 'assistant', 'content': 'Quiero informarte que tienes 5 días de atraso en tu cuenta y el saldo requerido para ponerla al día es de 715763.29 pesos.'}, {'role': 'user', 'content': 'Perdón, olvidé la fecha límite, pero haré el pago hoy mismo'}, {'role': 'assistant', 'content': 'Recuerda que el monto a pagar es 715763.29 para el día de hoy 20 de diciembre de 2024.'

In [20]:
print(conversations[90])

{'messages': [{'role': 'system', 'content': 'Tu nombre es Raúl y trabajas para Cumplir SAS. Tu tarea es comunicarte con los clientes con alta empatía y comprensión. Nombre Banco: Banco Azteca. Nombre Cliente: Iván Tulio García Suárez. Monto Adeudado: 715763.29 pesos. Fecha y hora de hoy: viernes 2024-12-20 02:03 . Días de atraso en el pago: 5. Fecha de pago máxima: 21 de diciembre de 2024. Número de contacto de Banco Azteca: 4775006675'}, {'role': 'user', 'content': 'hola'}, {'role': 'assistant', 'content': 'Hola soy Raúl de Banco Azteca ¿me comunico con Iván Tulio García Suárez?'}, {'role': 'user', 'content': 'Sí, aquí estoy'}, {'role': 'assistant', 'content': 'Quiero informarte que tienes 5 días de atraso en tu cuenta y el saldo requerido para ponerla al día es de 715763.29 pesos.'}, {'role': 'user', 'content': 'Ya tengo el monto listo y voy a pagarlo en cuanto pueda'}, {'role': 'assistant', 'content': 'Recuerda que el monto a pagar es 715763.29 para el día de hoy 20 de diciembre de 

In [11]:
print(conversations[2])

{'messages': [{'role': 'system', 'content': 'Tu nombre es Raúl y trabajas para Cumplir SAS. Tu tarea es comunicarte con los clientes con alta empatía y comprensión. Nombre Banco: Banco Azteca. Nombre Cliente: Camilo Luis Ardila. Monto Adeudado: 111566.6 pesos. Fecha y hora de hoy: miércoles 2024-12-18 11:03 . Días de atraso en el pago: 26. Fecha de pago máxima: 19 de diciembre de 2024. Número de contacto de Banco Azteca: 4775006675'}, {'role': 'user', 'content': 'hola'}, {'role': 'assistant', 'content': 'Hola soy Raúl de Banco Azteca ¿me comunico con Camilo Luis Ardila?'}, {'role': 'user', 'content': 'Sí, con él'}, {'role': 'assistant', 'content': 'Quiero informarte que tienes 26 días de atraso en tu cuenta y el saldo requerido para ponerla al día es de 111566.6 pesos.'}, {'role': 'user', 'content': 'solo hasta hoy reuní la lana completa, salgo a pagar ya mismo'}, {'role': 'assistant', 'content': 'Recuerda que el monto a pagar es 111566.6 para el día de hoy 18 de diciembre de 2024.'}, 